In [1]:
import json
import random


In [3]:
with open('./SquadFr-ranker.json',encoding="utf-8") as json_file:
                squadFR = json.load(json_file)

with open('./fquad-ranker.json',encoding="utf-8") as json_file:
                Fquad = json.load(json_file)
        
with open('./piaf-ranker.json',encoding="utf-8") as json_file:
                piaf = json.load(json_file)

In [8]:
def delete_repeated_questions(dataset):
    
    '''
    Delete repeated queries
    
    '''

    list_questions=[]
    for index,query in enumerate(dataset):
        question = query['question'] 
        if question in list_questions:
            dataset.pop(index)
        list_questions.append(question) 
    
    return dataset

In [13]:
def split_dataset(list_,train_ratio = 0.8):
    '''
    Split dataset to train,dev and test sets
    
    '''
    
    random.shuffle(list_)
    trainning_index = int(len(list_) * train_ratio)
    train = list_[:trainning_index]
    dev = list_[trainning_index:]
    
    return train,dev

In [5]:
len(squadFR),len(Fquad),len(piaf)

(84594, 20093, 7565)

In [6]:
dataset = squadFR + Fquad + piaf

In [9]:
dataset = delete_repeated_questions(dataset)

In [10]:
len(dataset)

112248

In [14]:
train,dev = split_dataset(dataset)

In [15]:
len(train),len(dev)

(89798, 22450)

In [17]:
with open('./squadFR_Fquad_piaf-train.json', 'w') as fp:
       json.dump(train, fp)

In [18]:
with open('./squadFR_Fquad_piaf-dev.json', 'w') as fp:
       json.dump(dev, fp)